# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0026'

In [3]:
pd.options.display.max_columns = 35

### Create Variables for Queries

In [4]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [5]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
       
""")
stdValue = stdVar.first()['std_length_of_stay']
print(stdValue)

12.8


In [6]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
 
      and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
             or v.vaccine_date is null) 
       
""")
stdValue_nonVac = stdVar.first()['std_length_of_stay']
print(stdValue_nonVac)

12.81


In [7]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null)        
      
    
""")
stdValue_vac = stdVar.first()['std_length_of_stay']
print(stdValue_vac)

5.62


In [8]:
spark.sql(f"""

          select
              tenant_zip_code
              , count(distinct h.personId) as count_of_non_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_nonVac}

              and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
                     or v.vaccine_date is null) 
                     
              and deceased = 1             
               
        group by 1
        order by 1
     
""").toPandas()

,tenant_zip_code,count_of_non_vaccinated_patients,mean_length_of_stay
0,0,1374,9.73
1,1,5513,10.83
2,2,2936,10.54
3,3,2269,10.74
4,4,846,8.83
5,5,743,9.43
6,6,1771,9.44
7,7,2332,10.40
8,8,4400,10.84
9,9,2683,10.58


In [9]:
spark.sql(f"""

        select count(distinct h.personId) as count_of_non_vaccinated_patients
               , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_nonVac}

              and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
                    or v.vaccine_date is not null)  
                    
              and deceased = 1            

""").toPandas()

,count_of_non_vaccinated_patients,mean_length_of_stay
0,31,11.24


In [10]:
spark.sql(f"""

          select
                tenant_zip_code
              , count(distinct h.personId) as count_of_vaccinated_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
                    
              and deceased = 1           
               
        group by 1
        order by 1
     
""").toPandas()

,tenant_zip_code,count_of_vaccinated_patients,mean_length_of_stay
0,5,2,4.50
1,6,2,6.00
2,7,2,4.67
3,8,4,4.80
4,9,1,2.00


In [11]:
spark.sql(f"""

          select count(distinct h.personId) as count_of_vaccinated_patients
                , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
               
              and deceased = 1       
      
""").toPandas()

,count_of_vaccinated_patients,mean_length_of_stay
0,11,4.5


In [12]:
spark.sql(f"""

          select
               tenant_zip_code
              , count(distinct h.personId) as count_of_all_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}
              
              and deceased = 1      

        group by 1
        order by 1
     
""").toPandas()

,tenant_zip_code,count_of_all_patients,mean_length_of_stay
0,0,1376,9.74
1,1,5513,10.83
2,2,2939,10.56
3,3,2270,10.74
4,4,846,8.83
5,5,746,9.43
6,6,1773,9.43
7,7,2335,10.40
8,8,4406,10.84
9,9,2684,10.58


In [13]:
spark.sql(f"""

        select count(distinct h.personId) as count_of_all_patients
              , round(mean(h.length_of_stay),2) as mean_length_of_stay
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue}

              and length(h.dischargeDate) > 3
              and length(h.serviceDate) > 3
            
              and deceased = 1      

     
""").toPandas()

,count_of_all_patients,mean_length_of_stay
0,24888,10.47
